In [1]:
import requests
import urllib
import html2text as ht
import re
import pandas as pd
import numpy as np
import json
import string
from bs4 import BeautifulSoup
from collections import Counter

## Scrape Billboard for song titles

In [2]:
class DateTracker:
    
    days = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 
            7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    
    def __init__(self, year = None, month = None, day = None):
        self.year = year
        self.month = month
        self.day = day
        
    def previous_week(self):
        
        if (self.year == 1958) and (self.month == 8) and (0 < self.day <= 7):
            return
        
        # if move to previous month
        if self.day - 7 <= 0:
            
            if self.month != 1:
                
                self.month -= 1
                self.day += self.days[self.month] - 7
            
            # if move to previous year
            else:
                
                self.year -= 1
                self.month = 12
                self.day += self.days[self.month] - 7
                
        else:
            
            self.day -= 7
            
    def as_date(self):
        
        return "%04d-%02d-%02d" % (self.year, self.month, self.day)
        
        
class Song:
    
    def __init__(self, artist, name, rank, peakpos, weeksonchart, date):
        self.artist = artist
        self.name = name
        self.rank = rank
        self.peakpos = peakpos
        self.weeksonchart = weeksonchart
        self.date = date
        
    def get_artist(self):
        return self.artist
    
    def get_name(self):
        return self.name
    
    def get_rank(self):
        return self.rank
    
    def get_peak_pos(self):
        return self.peakpos
    
    def get_weeks_on_chart(self):
        return self.weeksonchart
    
    def get_date(self):
        return self.date
    
    def list_form(self):
        return [self.artist, self.name, self.rank, self.peakpos,
                self.weeksonchart, self.date]
    
    def summary(self):
        print("\n".join([str(x) for x in [self.artist, self.name,
                                          self.rank, self.peakpos,
                                          self.weeksonchart, self.date]]))

In [3]:
def clean_text(text):
    bad = {"\\\\n": "\n", 
           "\n+": "\n", 
           "\n": " - ",
           "\[\]\(.{0,}?\)": "",
           "\[Play\]\(.{0,}?\)": "",
           "\[\s{0,5}-\s{0,5}Song\s{0,5}-\s{0,5}Lyrics\s{0,5}-\s{0,5}\]\(.{0,}?\)": ""}

    for x in bad:
        text = re.sub(x, bad[x], text, flags = re.I | re.S)
        
    return text

def clean_line(text):
    repl = {"weeks at no. 1": "",
            "(-\s{1,4}!)+": "",
            "(?<![A-z])! -": "",
            "_": "",
            "\[\s-": "", 
            "-\s\]": "",
            "\[.{0,}?\]|\(.{0,}?\)": "",
            "\s+": " ",
            "(-\s{1,5})+": "- ",}    
    
    for x in repl:
        text = re.sub(x, repl[x], text, flags = re.I | re.S)
    return text

In [79]:
def get_songs_by_week(weeks = 100):
    
    filler = [" and ", " featuring ", " & ", " x ", " / "]
    path = "https://www.billboard.com/charts/hot-100"
    curdate = DateTracker(year = 2018, month = 8, day = 11)
    songs = []

    for j in range(weeks):
        
        page = requests.get(path + "/" + curdate.as_date())
        if j % 20 == 0:
            print("We are on week {}, it is {}".format(j, curdate.as_date()))

        data = ht.html2text(str(page.content))
        data = clean_text(data)
        data = re.split("date search |in performance - |on chart - ", data, flags = re.I)[1:]

        #first one
        weeksoc = int(re.search("[0-9]{0,1}\s-\sweeks", 
                                re.sub("weeks at no. 1", "", data[0], flags = re.I | re.S),
                                flags = re.I | re.S).group()[:2])
        tmp = clean_line(data[1]).strip().split("-")
        name = tmp[0].strip()
        artist = tmp[1].strip()
        for k in filler:
            artist = re.sub(k, ", ", artist, flags = re.I | re.S)

        songs.append(Song(artist = artist, name = name, rank = 1,
                          peakpos = 1, weeksonchart = weeksoc, 
                          date = curdate.as_date()))


        #everything else (2 - 100)
        for i in data[2:]:
            #we know we have a rank
            if re.match("^[0-9]{0,3}", i.strip()).group() != "":
                i = clean_line(i)
                tmp = i.split("-")
                rank = tmp[0].strip()
                name = tmp[1].strip()
                artist = tmp[2].strip()
                for k in filler:
                    artist = re.sub(k, ", ", artist, flags = re.I | re.S)
                    
                #if these are empty they're debuts!
                try:
                    peakpos = int(re.search("[0-9]{0,3}\s-\speak position", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    peakpos = ""
                try:
                    weeksoc = int(re.search("[0-9]{0,3}\s-\sweeks", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    weeksoc = ""

                songs.append(Song(artist = artist, name = name, rank = rank,
                                  peakpos = peakpos, weeksonchart = weeksoc,
                                  date = curdate.as_date()))

        curdate.previous_week()
        
    return songs, curdate

In [80]:
songs, curdate = get_songs_by_week(weeks = 2)

We are on week 0, it is 2018-08-11


In [81]:
prep = np.array([x.list_form() for x in songs])
df = pd.DataFrame({
    "Artists": prep[:, 0],
    "Name": prep[:, 1],
    "Weekly rank": prep[:, 2],
    "Peak position": prep[:, 3],
    "Weeks on chart": prep[:, 4],
    "Week": prep[:, 5]
})

In [82]:
df.to_csv("Weekly_ranks.csv", header = True, index = False)

In [83]:
df.tail()

,Artists,Name,Peak position,Week,Weekly rank,Weeks on chart
193,"A$AP Rocky, Skepta",Praise The Lord,45,2018-08-04,96,7
194,NF,Lie,,2018-08-04,97,
195,Kane Brown,Lose It,84,2018-08-04,98,2
196,Drake,Emotionless,8,2018-08-04,99,4
197,Drake,Can't Take A Joke,18,2018-08-04,100,4


## Scrape Genius for lyrics, genre, etc.


Song name

Artist/Group

Weekly rank 

Peak rank

Year published

Genre

Writing Credits

Lyrics



In [91]:
def get_lyric_link(artists, name, both = False):
    
    """
    Returns the link of the song name given artists + name of the song
    """
    
    client_token = "BrGsH3KoiMzSyCUClF4-TyzjrNfQfrr2-Q9bfK6Bhum1fquRgVf0rn-Pq6mr9Uyc"
    headers = {"Authorization": "Bearer " + client_token}
    link = "https://api.genius.com/search?q="
    space = "%20"
    
    #this could prove disastrous, double check it...
    name_repl = {"Beyonce": "Beyoncé", 
                 "Amine": "Aminé",
                 "D.R.A.M.": "DRAM",
                 "$ign": "\$ign"}

    artists = re.sub(",", "", artists)
    name = re.sub(",", "", name)
    if both:
        page = requests.get(link + re.sub(" ", space, name) +
                            space + re.sub(" ", space, artists), headers = headers)
    else:
        page = requests.get(link + re.sub(" ", space, name), headers = headers)
        
    # data
    page = json.loads(page.content)["response"]["hits"] 
    check = [re.sub(",", "", x) for x in artists.split(" ") + name.split(" ") if x not in string.punctuation]
    
    for i in name_repl:
        if i in check:
            check.remove(i)
            check.append(name_repl[i])
    top = []

    if len(page) == 1:
        return page[0]["result"]["path"]
    else:
        for i in range(len(page)):
            c = 0
            
            #check if every artist + name token in the full title
            for j in check:
                if re.search(j, page[i]["result"]["full_title"], flags = re.I | re.S) != None:
                    c += 1
            if c == len(check):
                return page[i]["result"]["path"]

        
    #if we are here, search failed. now we include the artists as well
    if both == False:
        return get_lyric_link(artists, name, True)
    
    top = []
    
    #if here, find most popular song
    for j in range(len(page)):
        try:
            top.append(page[i]["result"]["stats"]["pageviews"])
        except KeyError:
            top.append(-1)
    print(top)
    return page[np.argmax(top)]["result"]["path"]





def scrape_lyrics(row, snip):
    
    #print(snip)
    base_link = "https://genius.com"
    content = requests.get(base_link + snip).content
    data = BeautifulSoup(content, "lxml").get_text()
    
    ##########################################
    
    # get all writers (expensive search)
    
    meta_data = content.decode("utf8")
    
    start_meta_regex = "verified_lyrics_by.*?writer_artists&quot"
    stop_meta_regex = "itemprop=\"page_data\""
    write_regex = "https://genius.com/artists/[^&]*"

    start = re.search(start_meta_regex, meta_data, flags = re.I | re.S).end()
    stop = re.search(stop_meta_regex, meta_data, flags = re.I | re.S).end()
    meta_data = meta_data[start:stop]

    write_links = re.findall(write_regex, meta_data)
    write = [re.sub("https://genius.com/artists/", "", x) for x in write_links]
    write = ", ".join([re.sub("-", " ", x) for x in write])
    
    
    ##########################################
    
    #subset data to search for lyrics
    
    genre_sub = {"\"": " ", "genius": "", "\s+,": ","}
    lyric_regex = row["Name"] + ".{0,20}?Lyrics.+?More on Genius"
    genre_regex = "genres\":\[.+?\]"
    track_info_regex = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?remixed by"
    track_info_regex2 = "\"{}".format(row["Name"]) + ".{0,20}?\".{0,20}?track info.+?cover by"
    date_regex = "release date.{0,15}?20[0-9][0-9]"
    write_regex = "Written By\n+.+?\n"
    
    data = data[re.search(base_link + snip, data).end():]
    
    lyrics = re.search(lyric_regex, data, flags = re.I | re.S).group()[:-15]
    
    try:
        genre = re.search("\[.+?\]", re.search(genre_regex, data, flags = re.I | re.S).group()).group()[1:-1]
        for k in genre_sub:
            genre = re.sub(k, genre_sub[k], genre, flags = re.I)
    except AttributeError:
        genre = ""
    

        
    try:
        date = re.split("\n+", re.search(date_regex, data, flags = re.I | re.S).group())[-1]
    except AttributeError:
        date = ""
        
    return date, genre, write, lyrics

In [92]:
dates = []
genre = []
write = []
lyrics = []
checker = {}

for i in range(len(df)):
    
    art = df.iloc[i]["Artists"]
    name = df.iloc[i]["Name"]
    
    # haven't scraped the song lyrics yet
    
    if art + name not in checker:
        
        temp = get_lyric_link(art, name)
        checker.update({art + name: i})

        try:
            d, g, w, l = scrape_lyrics(df.iloc[i], temp)
        except Exception:
            print(i)
            break
            
        dates.append(d)
        genre.append(g)
        write.append(w)
        lyrics.append(l)
    
    # already scraped song lyrics
    
    else:
        
        num = checker[art + name]
        d, g, w, l = dates[num], genre[num], write[num], lyrics[num]
        
        dates.append(d)
        genre.append(g)
        write.append(w)
        lyrics.append(l)        
    
    if i % 500 == 0:
        print("We are {}% done".format(round(i / len(df) * 100, 4)))

We are 0.0% done


error: multiple repeat at position 5